<a href="https://colab.research.google.com/github/aashutoshb97/CRISPR-AID-SD108/blob/master/unirep_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to use the UniRep mLSTM "babbler". This version demonstrates the 64-unit and the 1900-unit architecture. 

We recommend getting started with the 64-unit architecture as it is easier and faster to run, but has the same interface as the 1900-unit one.

Use the 64-unit or the 1900-unit model?

In [0]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

In [0]:
import awscli

In [3]:
pip install awscli

     |████████████████████████████████| 3.1MB 2.9MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 6.2MB 20.1MB/s 
  Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
  Found existing installation: botocore 1.16.19
    Uninstalling botocore-1.16.19:
      Successfully uninstalled botocore-1.16.19


In [1]:
pip install tensorflow==1.3.0

     |████████████████████████████████| 43.6MB 93kB/s 
     |████████████████████████████████| 1.6MB 37.7MB/s 
     |████████████████████████████████| 890kB 42.7MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=d6e68c6bddf030e40ed8f96705320bafd99f772961a8d71f83161f0149401519
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


## Setup

In [4]:
import tensorflow as tf
import numpy as np

# Set seeds
tf.set_random_seed(42)
#tf.random.set_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarning: Passing (type, 1) or 

## Data formatting and management

Initialize UniRep, also referred to as the "babbler" in our code. You need to provide the batch size you will use and the path to the weight directory.

In [0]:
batch_size = 403
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

UniRep needs to receive data in the correct format, a (batch_size, max_seq_len) matrix with integer values, where the integers correspond to an amino acid label at that position, and the end of the sequence is padded with 0s until the max sequence length to form a non-ragged rectangular matrix. We provide a formatting function to translate a string of amino acids into a list of integers with the correct codex:

In [0]:
seq = "MRKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

In [7]:
np.array(b.format_seq(seq))

array([24,  1,  2,  4, 13,  6,  6, 21, 18,  8, 13, 16, 16, 14, 17, 21, 16,
        6, 21,  5, 13,  5, 16,  9, 13,  3,  4, 18,  7, 16,  2, 13,  6, 13,
        6, 13,  5, 15,  8,  9, 13,  4, 21,  8, 21,  4, 18, 17, 11,  8,  8,
       13,  4, 21, 14, 16, 14, 20, 14,  8, 21, 16,  8,  8, 21,  8, 19, 13,
       16, 10, 11, 18, 15,  2, 19, 14,  5,  3,  1,  4, 10,  3,  5, 18, 18,
        4,  7, 15,  1, 14,  6, 13, 19, 16, 10,  6,  2,  8, 17,  7, 18,  4,
        5,  5, 13,  8, 19,  4,  8,  2, 15,  6, 16,  4, 18,  6, 13,  5,  8,
       21, 16,  9,  2, 17,  6, 21,  4, 13, 17,  5, 18,  4,  6,  5, 13,  9,
       17, 21, 13,  3,  4, 21,  6, 19,  9, 18,  9,  7,  3,  9, 16, 19, 17,
        8, 15,  5,  4, 10,  4,  9, 13, 17,  4, 15,  9, 18,  4, 17,  2,  3,
        9, 16,  6,  5, 13,  7, 16, 10, 21, 15,  5,  3, 19, 10, 10,  9,  8,
       14, 17, 13,  5, 13, 14, 16, 21, 21, 14,  5,  9,  3, 19, 21,  7,  8,
       10,  7, 16, 21,  7,  4,  5, 14,  9,  6,  4,  2,  5,  3,  1, 16, 21,
       21,  6, 18, 16,  8

We also provide a function that will check your amino acid sequences don't contain any characters which will break the UniRep model.

In [8]:
b.is_valid_seq(seq)

True

You could use your own data flow as long as you ensure that the data format is obeyed. Alternatively, you can use the data flow we've implemented for UniRep training, which happens in the tensorflow graph. It reads from a file of integer sequences, shuffles them around, collects them into groups of similar length (to minimize padding waste) and pads them to the max_length. Here's how to do that:

First, sequences need to be saved in the correct format. Suppose we have a new-line seperated file of amino acid sequences, `seqs.txt`, and we want to format them. Note that training is currently only publicly supported for amino acid sequences less than 275 amino acids as gradient updates for sequences longer than that start to get unwieldy. If you want to train on sequences longer than this, please reach out to us. 

Sequence formatting can be done as follows:

In [32]:
# Before you can train your model, 
y_output = []
with open("non_log_int_output.txt", "r") as source:
  for i in enumerate(source):
    y_output.append(i[1][:5].strip("\n"))

print(y_output)
#print(type(y_output))
with open("seqs.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            if b.is_valid_seq(seq) and len(seq) < 2500: 
                formatted_input = ",".join(map(str,b.format_seq(seq)))
                #print(type(formatted_input))
                formatted = y_output[i] + "," + formatted_input
                #print(formatted)
                destination.write(formatted)
                destination.write('\n')

    

['54', '88', '73', '84', '62', '42', '14', '16', '39', '42', '47', '124', '149', '73', '38', '50', '110', '64', '27', '43', '8', '28', '86', '98', '7', '113', '158', '83', '12', '55', '36', '79', '38', '55', '111', '68', '217', '74', '35', '123', '18', '90', '76', '23', '39', '84', '300', '51', '65', '159', '91', '30', '26', '55', '54', '61', '88', '61', '290', '10', '81', '23', '114', '60', '282', '51', '108', '17', '48', '111', '26', '76', '68', '61', '45', '93', '82', '206', '38', '107', '67', '46', '62', '33', '63', '89', '72', '130', '105', '146', '73', '29', '50', '10', '38', '14', '42', '41', '156', '23', '43', '133', '73', '122', '40', '47', '65', '92', '86', '25', '13', '108', '77', '96', '452', '81', '16', '21', '9', '58', '73', '68', '144', '50', '56', '81', '44', '47', '105', '136', '159', '40', '230', '35', '58', '109', '104', '84', '105', '118', '81', '96', '109', '105', '104', '91', '48', '105', '78', '154', '44', '148', '171', '51', '38', '95', '29', '6', '20', '53', '4

This is what the integer format looks like

In [33]:
!head -n1 formatted.txt

54,24,1,18,4,16,14,16,13,21,15,7,2,8,2,6,21,1,9,7,16,8,21,9,7,21,9,9,13,4,13,18,9,1,19,21,14,13,17,21,2,15,18,14,4,14,16,14,7,15,17,8,7,14,15,17,14,4,19,2,13,6,7,18,10,18,2,4,21,7,11,17,7,7,9,19,11,7,8,8,3,10,18,21,7,7,21,4,7,7,8,7,2,21,16,13,4,2,15,18,3,7,7,2,2,15,6,17,4,18,17,18,7,7,4,7,14,4,9,13,9,4,14,18,16,4,16,19,4,16,7,14,18,18,17,17,18,15,8,15,7,17,18,8,18,17,21,8,7,8,17,16,16,17,14,21,17,18,3,18,18,18,14,21,21,17,1,18,18,18,18,4,10,18,4,4,20,10,4,9,17,18,19,4,5,16,21,8,7,21,14,4,8,4,21,4,17,8,21,14,8,1,2,7,21,10,21,10,14,1,16,10,7,20,4,6,17,7,7,2,1,13,17,14,9,6,18,15,4,13,21,9,16,5,21,16,4,10,6,6,8,2,4,10,18,21,7,18,21,10,4,2,16,21,6,7,18,8,4,9,6,21,13,17,2,7,19,18,21,13,5,7,16,6,4,20,17,4,6,7,19,5,21,6,21,5,17,5,9,11,2,7,6,21,2,4,18,10,8,18,17,18,7,7,16,2,19,4,21,19,21,5,7,1,4,9,21,14,21,9,14,7,4,4,21,6,13,4,4,3,17,15,5,16,19,16,17,17,21,5,6,7,18,14,15,17,1,18,9,13,13,15,19,7,4,15,5,18,18,4,17,21,10,6,7,6,8,7,9,7,7,4,8,21,9,8,17,17,15,17,4,7,16,9,8,21,21,7,4,3,18,16,17,8,8,9,

Notice that by default format_seq does not include the stop symbol (25) at the end of the sequence. This is the correct behavior if you are trying to train a top model, but not if you are training UniRep representations.

Now we can use a custom function to bucket, batch and pad sequences from `formatted.txt` (which has the correct integer codex after calling `babbler.format_seq()`). The bucketing occurs in the graph. 

What is bucketing? Specify a lower and upper bound, and interval. All sequences less than lower or greater than upper will be batched together. The interval defines the "sides" of buckets between these bounds. Don't pick a small interval for a small dataset because the function will just repeat a sequence if there are not enough to
fill a batch. All batches are the size you passed when initializing the babbler.

This is also doing a few other things:
- Shuffling the sequences by randomly sampling from a 10000 sequence buffer
- Automatically padding the sequences with zeros so the returned batch is a perfect rectangle
- Automatically repeating the dataset

In [34]:
bucket_op_f = b.bucket_batch_pad("formatted.txt", interval=1000) # Large interval
print(type(bucket_op_f))

<class 'tensorflow.python.framework.ops.Tensor'>


Inconveniently, this does not make it easy for a value to be associated with each sequence and not lost during shuffling. You can get around this by just prepending every integer sequence with the sequence label (eg, every sequence would be saved to the file as "{brightness value}, 24, 1, 5,..." and then you could just index out the first column after calling the `bucket_op`. Please reach out if you have questions on how to do this.

In [0]:
#indexing out first column after calling bucket_op
#print(type(tf.Session().run(bucket_op_f)))
#print(tf.Session().run(bucket_op_f)[:,1:]) 
y_output_final = tf.Session().run(bucket_op_f)[:,0]
bucket_op = tf.convert_to_tensor(tf.Session().run(bucket_op_f)[:,1:])

Now that we have the `bucket_op`, we can simply `sess.run()` it to get a correctly formatted batch

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = sess.run(bucket_op)

You can look back and see that the batch_size we passed to __init__ is indeed 12, and the second dimension must be the longest sequence included in this batch. Now we have the data flow setup (note that as long as your batch looks like this, you don't need my flow), so we can proceed to implementing the graph. The module returns all the operations needed to feed in sequence and get out trainable representations.

## Training a top model and a top model + mLSTM.

First, obtain all of the ops needed to output a representation

In [0]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())

`final_hidden` should be a batch_size x rep_dim matrix.

Lets say we want to train a basic feed-forward network as the top model, doing regression with MSE loss, and the Adam optimizer. We can do that by:

1.  Defining a loss function.

2.  Defining an optimizer that's only optimizing variables in the top model.

3.  Minimizing the loss inside of a TensorFlow session

In [0]:
#y_placeholder = tf.placeholder(tf.float32, shape=[None,1], name="y")
y_placeholder = tf.convert_to_tensor(y_output_final)
'''
with tf.variable_scope("top"):
    prediction = tf.contrib.layers.fully_connected(
        final_hidden, 1, activation_fn=None, 
        weights_initializer=initializer,
        biases_initializer=tf.zeros_initializer()
    )
'''


loss = tf.losses.mean_squared_error(y_placeholder, prediction)

You can specifically train the top model first by isolating variables of the "top" scope, and forcing the optimizer to only optimize these.

In [0]:
learning_rate=.001
top_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="top")
optimizer = tf.train.AdamOptimizer(learning_rate)
top_only_step_op = optimizer.minimize(loss, var_list=top_variables)
all_step_op = optimizer.minimize(loss)

We next need to define a function that allows us to calculate the length each sequence in the batch so that we know what index to use to obtain the right "final" hidden state

In [0]:
def nonpad_len(batch):
    nonzero = batch > 0
    lengths = np.sum(nonzero, axis=1)
    return lengths

nonpad_len(batch)

array([239, 248, 205, 111,  97, 223, 148, 239, 232, 124, 120, 230])

We are ready to train. As an illustration, let's learn to predict the number 42 just optimizing the top model.

In [0]:
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, top_only_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
                  
        print("Iteration {0}: {1}".format(i, loss_))

Iteration 0: 1727.8043212890625
Iteration 1: 1726.61083984375
Iteration 2: 1725.66650390625
Iteration 3: 1722.8544921875
Iteration 4: 1724.3453369140625
Iteration 5: 1722.4638671875
Iteration 6: 1720.9686279296875
Iteration 7: 1720.2288818359375
Iteration 8: 1717.4027099609375
Iteration 9: 1718.5859375


We can also jointly train the top model and the mLSTM. Note that if using the 1900-unit (full) model, you will need a GPU with at least 16GB RAM. To see a demonstration of joint training with fewer computational resources, please run this notebook using the 64-unit model.

In [0]:
y = [[42]]*batch_size
num_iters = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, all_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
        
        print("Iteration {0}: {1}".format(i,loss_))

Iteration 0: 1727.8043212890625
Iteration 1: 1703.43408203125
Iteration 2: 1678.0831298828125
Iteration 3: 1642.8365478515625
Iteration 4: 1620.3525390625
Iteration 5: 1580.65771484375
Iteration 6: 1546.9124755859375
Iteration 7: 1504.5289306640625
Iteration 8: 1459.1434326171875
Iteration 9: 1432.6119384765625
